In [ ]:
# from simd_r_drive import DataStore
from us_gaap_store import UsGaapStore

from simd_r_drive_ws_client import DataStoreWsClient

WEBSOCKET_ADDRESS = "127.0.0.1:58086"
data_store = DataStoreWsClient(WEBSOCKET_ADDRESS)
us_gaap_store = UsGaapStore(data_store)

In [ ]:
sample_embedding = us_gaap_store.lookup_by_index(0)["embedding"]

sample_embedding.shape


In [ ]:
# TODO: Move to preprocessing

# # UMAP visualization
# umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, metric="cosine")
# umap_2d = umap_model.fit_transform(compressed)

# plt.figure(figsize=(10, 6))
# plt.scatter(umap_2d[:, 0], umap_2d[:, 1], c=labels, cmap="tab10", s=5)
# plt.title("Concept/UOM Embeddings Clustered")
# plt.show()

In [ ]:
# df = pd.DataFrame({
#     "concept": [c for c, _ in concept_unit_pairs],
#     "unit": [u for _, u in concept_unit_pairs],
#     "cluster": labels
# })
# grouped = df.groupby("cluster")

# for cluster_id, group in grouped:
#     print(f"\nCluster {cluster_id} ({len(group)} items):")
#     print(group.head(10).to_string(index=False))

# noise = df[df["cluster"] == -1]

# print(f"Noise points: {len(noise)}")


In [ ]:
# noise_points = df[df["cluster"] == -1][["concept", "unit"]].reset_index(drop=True)

# noise_points.to_csv("noise_points.csv")

In [ ]:
# import numpy as np

# # Save both embeddings and tuples
# np.savez_compressed(
#     "data/stage1_latents.npz",
#     keys=np.array([f"{c}::{u}" for c, u in concept_unit_pairs]),
#     embeddings=compressed,
#     concept_unit_value_tuples=np.array(concept_unit_value_tuples, dtype=object)
# )

# print(f"✅ Saved {len(concept_unit_value_tuples):,} tuples and {len(compressed):,} embeddings to 'stage1_latents.npz'")


In [ ]:
# TODO: Move

# # Debug scaled stats

# from collections import defaultdict
# import numpy as np

# # Reconstruct per (concept, unit) from scaled_tuples
# reconstructed = defaultdict(list)
# all_vals = []

# for concept, unit, val in scaled_tuples:
#     key = (concept, unit)
#     reconstructed[key].append(val)
#     all_vals.append(val)

# total_items = 0
# total_no_variance_items = 0

# # Analyze per group
# for key, vals in reconstructed.items():
#     vals_np = np.array(vals)

#     if not np.all(np.isfinite(vals_np)):
#         print(f"[BAD SCALE] {key} has non-finite scaled values!")
#         print("  Sample:", vals_np[:5])
#         continue

#     max_val = np.max(vals_np)
#     min_val = np.min(vals_np)
#     mean_val = np.mean(vals_np)

#     if max_val > 1e6 or min_val < -1e6:
#         print(f"[OUTLIER SCALE] {key} range is extreme:")
#         print("  min:", min_val, "max:", max_val, "mean:", mean_val)

#     if np.allclose(max_val, min_val):
#         # print(f"[FLAT SCALE] {key} has no variance ({len(vals)} item(s)):")
#         # print("  value:", max_val)
#         total_no_variance_items += len(vals)

#     total_items += len(vals)

# # Global stats
# all_vals_np = np.array(all_vals)

# print("\n=== GLOBAL STATS ===")
# print(f"Global max: {np.max(all_vals_np)}")
# print(f"Global min: {np.min(all_vals_np)}")
# print(f"Global mean: {np.mean(all_vals_np)}")
# print(f"Global std: {np.std(all_vals_np)}")

# print(f"Total items: {total_items}")
# print(f"Total no variance items: {total_no_variance_items}")


In [ ]:
from models.pytorch.narrative_stack.stage1.dataset import IterableConceptValueDataset, collate_with_scaler
from models.pytorch.narrative_stack.stage1 import Stage1Autoencoder

In [ ]:
# Tuning

# import os
# import optuna
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
# from torch.utils.data import DataLoader
# from utils.pytorch import get_device

# device = get_device()

# # === CONFIG ===
# OUTPUT_PATH = "data/stage1"
# os.makedirs(OUTPUT_PATH, exist_ok=True)
# OPTUNA_DB_PATH = os.path.join(OUTPUT_PATH, "optuna_study.db")
# EPOCHS = 3
# PATIENCE = 5
# VAL_SPLIT = 0.2

# def objective(trial):
#     batch_size = trial.suggest_int("batch_size", 8, 64, step=8)
#     lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
#     latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
#     dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.2, step=0.1)
#     weight_decay = trial.suggest_float("weight_decay", 1e-8, 1e-4, log=True)
#     gradient_clip = trial.suggest_float("gradient_clip", 0.0, 1.0, step=0.1)

#     # # 80/20 Train/Val Split
#     # split = int(len(scaled_tuples) * (1 - VAL_SPLIT))
#     # train_data = scaled_tuples[:split]
#     # val_data = scaled_tuples[split:]

#      # === Sample Subset for Faster Debugging ===
#     SAMPLE_SIZE = 500_000
#     subset = scaled_tuples[:SAMPLE_SIZE]

#     # 80/20 Train/Val Split
#     split = int(len(subset) * (1 - VAL_SPLIT))
#     train_data = subset[:split]
#     val_data = subset[split:]

#     train_loader = DataLoader(
#         ConceptValueDataset(train_data, embedding_map, device=device, value_noise_std=0.005, train=True),
#         batch_size=batch_size,
#         shuffle=True
#     )

#     val_loader = DataLoader(
#         ConceptValueDataset(val_data, embedding_map, device=device, value_noise_std=0.00, train=False),
#         batch_size=batch_size,
#         shuffle=False
#     )

#     input_dim = len(next(iter(embedding_map.values()))) + 1

#     model = Stage1Autoencoder(
#         input_dim=input_dim,
#         latent_dim=latent_dim,
#         dropout_rate=dropout_rate,
#         lr=lr,
#         batch_size=batch_size,
#         weight_decay=weight_decay,
#         gradient_clip=gradient_clip
#     )

#     early_stop_callback = EarlyStopping(monitor="val_loss", patience=PATIENCE, verbose=True, mode="min")

#     model_checkpoint = ModelCheckpoint(
#         dirpath=OUTPUT_PATH,
#         filename="best_model_trial_{trial.number}",
#         monitor="val_loss",
#         mode="min",
#         save_top_k=1,
#         verbose=True
#     )

#     trainer = pl.Trainer(
#         max_epochs=EPOCHS,
#         logger=TensorBoardLogger(OUTPUT_PATH, name="stage1_autoencoder"),
#         callbacks=[early_stop_callback, model_checkpoint],
#         accelerator="auto",
#         devices=1,
#         gradient_clip_val=gradient_clip
#     )

#     trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
#     return trainer.callback_metrics["val_loss"].item()

# # === Optuna Study ===
# study = optuna.create_study(direction="minimize",
#                             storage=f"sqlite:///{OPTUNA_DB_PATH}",
#                             load_if_exists=True)
# study.optimize(objective, n_trials=25)

# print("Best params:", study.best_params)
# print("Best trial value:", study.best_trial.value)


In [ ]:
# from torch.utils.data import DataLoader

# # Instantiate dataset
# dataset = ConceptValueDataset(scaled_tuples, embedding_map)

# # Sample inspection
# sample_x, sample_y = dataset[0]
# print("Sample input:", sample_x)
# print("Min:", sample_x.min().item(), "Max:", sample_x.max().item())
# print("Mean:", sample_x.mean().item(), "Std:", sample_x.std().item())
# print("Input dim:", sample_x.shape[0], "Target dim:", sample_y.shape[0])

# # Optional: test batch loading
# loader = DataLoader(dataset, batch_size=4)
# for xb, yb in loader:
#     print("Batch shape:", xb.shape)
#     break


In [ ]:
# train_data = list(scaled_tuples)
# val_data = train_data

# # TODO: Get rid of most of this and just evaluate on the full train_data set

# from collections import defaultdict
# from sklearn.utils import shuffle
# from sklearn.model_selection import train_test_split # Make sure this is imported
# import numpy as np # For potential use, though train_test_split handles counts

# # Target fraction for the validation set from each group
# VALIDATION_FRACTION = 0.25
# RANDOM_STATE = 42

# # Assume `scaled_tuples` is the list of (concept, unit, scaled_value)
# # generated by your previous preprocessing script. It contains ALL your scaled data.

# # 1. Training data will be 100% of scaled_tuples.
# # We create a list copy for clarity, though direct use is also possible.
# # The DataLoader for training will typically shuffle this.
# train_data = list(scaled_tuples)

# # 2. Validation data will be a stratified sample, aiming for ~25% of each group's items.
# val_data = []

# # Group all tuples by (concept, unit) to perform stratified sampling for the validation set.
# # This grouping is based on the full scaled_tuples list.
# grouped_for_val_sampling = defaultdict(list)
# for concept, unit, value in scaled_tuples: # Iterate over the full dataset
#     grouped_for_val_sampling[(concept, unit)].append(value)

# # For each group, select a VALIDATION_FRACTION sample of its values to contribute to val_data.
# # These samples will also be part of the 100% train_data.
# for (concept, unit), values_in_group in grouped_for_val_sampling.items():
#     if not values_in_group: # Skip if group is empty
#         continue

#     num_samples_in_group = len(values_in_group)
#     group_val_values = [] # To store validation values from this specific group

#     if num_samples_in_group == 1:
#         # If a group has only one sample, and we want validation data (VALIDATION_FRACTION > 0),
#         # we include this single sample in the validation set.
#         # This means all single-sample groups will be represented in this stratified validation set.
#         if VALIDATION_FRACTION > 0:
#             group_val_values = list(values_in_group) # Take the single value
#     elif num_samples_in_group > 1:
#         # For groups with more than one sample, use train_test_split to get
#         # the desired fraction for the validation set from this group.
#         # The '_group_train_dummy' part is not used for constructing val_data here.
#         # We ensure shuffling before split if not done by train_test_split, but it shuffles by default.
#         _group_train_dummy, sampled_val_values_for_group = train_test_split(
#             values_in_group,       # Values from the current (concept, unit) group
#             test_size=VALIDATION_FRACTION,
#             random_state=RANDOM_STATE,
#             shuffle=True          # Ensure shuffling for random sampling within the group
#         )
#         group_val_values = sampled_val_values_for_group

#     # Extend the global val_data list with the (concept, unit, value) tuples from this group's sample
#     if group_val_values: # If any values were selected for validation from this group
#         val_data.extend([(concept, unit, v) for v in group_val_values])

# print(f"Total items for training (100%): {len(train_data)}")
# print(f"Total items for validation (stratified ~25% sample): {len(val_data)}")

# # Now you have:
# # - `train_data`: A list of all your (concept, unit, scaled_value) tuples (100%).
# # - `val_data`: A list containing a stratified sample, where each (concept, unit) group
# #               contributes approximately 25% of its items to this validation set.
# #               The total size will be roughly 25% of the original dataset, but may vary
# #               slightly due to per-group integer rounding and handling of small groups.

# # You would then use `train_data` and `val_data` in your PyTorch DataLoaders.
# # The `train_loader` will use `train_data` (100%) with shuffle=True.
# # The `val_loader` will use `val_data` (the ~25% stratified sample) with shuffle=False.

In [ ]:
# For debugging, only

# Example: keep only 10,000 training and 2,000 validation samples
# train_data = train_data[:100_000]
# val_data = val_data[:2_000]
# val_data = train_data

# print(f"Truncated train_data: {len(train_data)}")
# print(f"Truncated val_data: {len(val_data)}")

In [ ]:
# import torch
# from utils.pytorch import get_device # Assuming you have this

# device = get_device() # Make sure device is defined

# # Define Median Scaled Value (which is 0.0 after RobustScaler)
# median_scaled_val_tensor = torch.tensor(0.0, dtype=torch.float32, device=device)
# print(f"Using median_scaled_val: {median_scaled_val_tensor.item()}")

# # === Calculate Mean Embedding (mean_emb) ===
# # (Calculation for mean_emb_tensor remains the same as before)
# # Get all unique (concept, unit) keys from train_data
# train_keys = set((item[0], item[1]) for item in train_data)
# # Get the corresponding embeddings
# train_embeddings = [embedding_map[key] for key in train_keys if key in embedding_map]

# if not train_embeddings:
#     raise ValueError("No valid embeddings found for training data keys!")

# # Stack embeddings into a numpy array and calculate the mean vector
# train_embeddings_np = np.stack(train_embeddings)
# mean_emb_numpy = np.mean(train_embeddings_np, axis=0)
# # Convert to a tensor on the correct device
# mean_emb_tensor = torch.tensor(mean_emb_numpy, dtype=torch.float32, device=device)
# print(f"Calculated mean_emb shape: {mean_emb_tensor.shape}")
# print(f"Calculated mean_emb value (first 10 elements): {mean_emb_tensor[:10]}")

In [ ]:
# Training

import os
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.data import DataLoader
from utils.pytorch import get_device
import functools

device = get_device()

# === CONFIG ===
OUTPUT_PATH = "data/stage1_23_(no_pre_dedupe)"
os.makedirs(OUTPUT_PATH, exist_ok=True)
OPTUNA_DB_PATH = os.path.join(OUTPUT_PATH, "optuna_study.db")
EPOCHS = 1000
PATIENCE = 20  # Compensate for long annealing period + some

# ckpt_path = f"{OUTPUT_PATH}/stage1_resume-v8.ckpt"
# ckpt_path = f"{OUTPUT_PATH}/manual_resumed_checkpoint.ckpt"
ckpt_path = None

# model = Stage1Autoencoder.load_from_checkpoint(ckpt_path,
#     # lr=5e-5,
#     lr=2.5e-6,
#     # min_lr=1e-6,
#     min_lr=1e-7
# )
model = Stage1Autoencoder()

batch_size = model.hparams.batch_size
gradient_clip = model.hparams.gradient_clip

# train_loader = DataLoader(
#     ConceptValueDataset(train_data, embedding_map, device=device, value_noise_std=0.005, train=True),
#     batch_size=batch_size,
#     shuffle=True
# )

# val_loader = DataLoader(
#     ConceptValueDataset(val_data, embedding_map, device=device, value_noise_std=0.00, train=False),
#     batch_size=batch_size,
#     shuffle=False
# )
train_loader = DataLoader(
    IterableConceptValueDataset(WEBSOCKET_ADDRESS, internal_batch_size=16, return_scaler=True, shuffle=True),
    batch_size=batch_size,
    # shuffle=True, # Moved to dataset
    collate_fn=collate_with_scaler,
    pin_memory=True,
    persistent_workers=True,
    num_workers=1,
)

val_loader = DataLoader(
    IterableConceptValueDataset(WEBSOCKET_ADDRESS, internal_batch_size=16, return_scaler=True, shuffle=False),
    batch_size=batch_size,
    # shuffle=False, # Moved to dataset
    collate_fn=collate_with_scaler,
    pin_memory=True,
    persistent_workers=True,
    num_workers=1,
)

# input_dim = len(next(iter(embedding_map.values()))) + 1
sample_embedding = us_gaap_store.lookup_by_index(0)["embedding"]
input_dim = sample_embedding.shape[0]

early_stop_callback = EarlyStopping(
    monitor="val_loss_epoch", patience=PATIENCE, verbose=True, mode="min"
)

model_checkpoint = ModelCheckpoint(
    dirpath=OUTPUT_PATH,
    filename="stage1_resume",
    monitor="val_loss_epoch",
    mode="min",
    save_top_k=1,
    verbose=True,
)

trainer = pl.Trainer(
    max_epochs=EPOCHS,
    logger=TensorBoardLogger(OUTPUT_PATH, name="stage1_autoencoder"),
    callbacks=[early_stop_callback, model_checkpoint],
    accelerator="auto",
    devices=1,
    gradient_clip_val=gradient_clip,
)

trainer.fit(
    model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    #
    # ckpt_path=ckpt_path # TODO: Uncomment if resuming training AND wanting to restore existing model configuration
)

In [ ]:
# trainer.save_checkpoint(f"{OUTPUT_PATH}/manual_resumed_checkpoint.ckpt")

# Conceptual Draft

Stage 1 learns semantic+quantitative embeddings for individual concept/unit/value triplets.

Stage 2 learns how to aggregate and contextualize those embeddings into higher-order units (i.e., financial statements).

Stage 3 learns how to model temporal dynamics and structural evolution across filings — a full hierarchy of understanding.

This pipeline could encode an entire company's financial narrative into vector space.

It’s structured like language modeling, but for accounting — and that’s what makes it potentially groundbreaking.

In [ ]:
# import numpy as np
# import torch

# import numpy as np
# import torch

# # Where correlation matrix is on the full z, and the `corr_value` is derived specifically from the input value dimension
# # IMPORTANT: This should only be used with this "stage 1" model
# def analyze_latent_correlation_matrix_streaming(model, val_loader, device):
#     model.eval()
#     model.to(device)

#     latent_dim = model.hparams.latent_dim

#     count = 0
#     mean_z = np.zeros(latent_dim)
#     m2_z = np.zeros(latent_dim)
#     cov_z = np.zeros((latent_dim, latent_dim))

#     # For value correlation
#     mean_val = 0.0
#     m2_val = 0.0
#     cov_val = np.zeros(latent_dim)

#     for batch in val_loader:
#         x, y, _ = batch
#         x = x.to(device)
#         y = y.to(device)

#         z = model.encode(x).detach().cpu().numpy()  # [B, D]
#         v = y[:, -1].detach().cpu().numpy()         # [B]

#         for zi, vi in zip(z, v):
#             count += 1

#             # === Update latent stats (Welford) ===
#             delta_z = zi - mean_z
#             mean_z += delta_z / count
#             m2_z += delta_z * (zi - mean_z)

#             # === Update cov_z (outer product) ===
#             cov_z += np.outer(delta_z, zi - mean_z)

#             # === Update value stats ===
#             delta_v = vi - mean_val
#             mean_val += delta_v / count
#             m2_val += delta_v * (vi - mean_val)

#             # === Update cov_val ===
#             cov_val += delta_z * (vi - mean_val)

#         # break

#     var_z = m2_z / (count - 1)
#     var_val = m2_val / (count - 1)
#     cov_z /= (count - 1)
#     cov_val /= (count - 1)

#     std_z = np.sqrt(var_z + 1e-8)
#     std_val = np.sqrt(var_val + 1e-8)

#     corr_matrix = cov_z / (std_z[:, None] * std_z[None, :])
#     corr_value = cov_val / (std_z * std_val)

#     return corr_matrix, corr_value


# print("\n=== Computing latent correlation matrix... ===")
# corr_matrix, corr_value = analyze_latent_correlation_matrix_streaming(model, val_loader, device=device)

# import matplotlib.pyplot as plt

# def plot_correlation_matrix(corr_matrix, title="Latent Dimension Correlation Matrix"):
#     plt.figure(figsize=(10, 8))
#     plt.imshow(corr_matrix, cmap='coolwarm', interpolation='nearest', vmin=-1, vmax=1)
#     plt.colorbar(shrink=0.5)
#     plt.title(title)
#     plt.xlabel("Latent Dim")
#     plt.ylabel("Latent Dim")
#     plt.tight_layout()
#     plt.show()

# plot_correlation_matrix(corr_matrix)

# top_k = 128

# def print_top_latent_correlations(corr_matrix, top_k=top_k):
#     dim = corr_matrix.shape[0]
#     pairs = []

#     for i in range(dim):
#         for j in range(i + 1, dim):
#             corr = corr_matrix[i, j]
#             pairs.append(((i, j), corr))

#     top_corrs = sorted(pairs, key=lambda x: -abs(x[1]))[:top_k]

#     print(f"\nTop {top_k} most correlated latent dimension pairs:")
#     for (i, j), corr in top_corrs:
#         print(f"z[{i:03d}] ↔ z[{j:03d}]: corr = {corr:.4f}")


# print_top_latent_correlations(corr_matrix, top_k=top_k)


# top_dims = sorted(enumerate(corr_value), key=lambda x: -abs(x[1]))[:top_k]
# print("\nTop latent dimensions most correlated with scaled value:")
# for i, c in top_dims:
#     print(f"z[{i:03d}]: corr = {c:.4f}")


# plt.figure(figsize=(12, 4))
# plt.plot(np.sort(np.abs(corr_value))[::-1], marker='o')
# plt.title("Absolute Correlation of Latent Dims with Value")
# plt.xlabel("Sorted Latent Dimension")
# plt.ylabel("Absolute Correlation")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

